In [1]:
import boto3
ec2_cli = boto3.client('ec2')
print(ec2_cli.describe_regions())

{'Regions': [{'Endpoint': 'ec2.eu-north-1.amazonaws.com', 'RegionName': 'eu-north-1', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.ap-south-1.amazonaws.com', 'RegionName': 'ap-south-1', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.eu-west-3.amazonaws.com', 'RegionName': 'eu-west-3', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.eu-west-2.amazonaws.com', 'RegionName': 'eu-west-2', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.eu-west-1.amazonaws.com', 'RegionName': 'eu-west-1', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.ap-northeast-2.amazonaws.com', 'RegionName': 'ap-northeast-2', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.ap-northeast-1.amazonaws.com', 'RegionName': 'ap-northeast-1', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.sa-east-1.amazonaws.com', 'RegionName': 'sa-east-1', 'OptInStatus': 'opt-in-not-required'}, {'Endpoint': 'ec2.ca-central-1.amazonaws.com', 'RegionName': 'ca-central-1', 'OptIn

In [ ]:
import pprint
pprint.pprint(ec2_cli.describe_regions())

In [3]:
for region in ec2_cli.describe_regions()['Regions']:
    print(region['RegionName'])

eu-north-1
ap-south-1
eu-west-3
eu-west-2
eu-west-1
ap-northeast-2
ap-northeast-1
sa-east-1
ca-central-1
ap-southeast-1
ap-southeast-2
eu-central-1
us-east-1
us-east-2
us-west-1
us-west-2


In [ ]:
ec2client = boto3.client('ec2')
response = ec2client.describe_instances()
for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        pprint.pprint(instance)

In [5]:
ec2client = boto3.client('ec2')
response = ec2client.describe_instances()
for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        try:
            print(instance['InstanceId'], instance['CpuOptions'], instance['Tags']) #instance['Tags']
        except:
            print()

i-0b816295697f971fd {'CoreCount': 1, 'ThreadsPerCore': 1} [{'Key': 'Name', 'Value': 'docker_test'}]


In [6]:
ec2client = boto3.client('ec2')
response = ec2client.describe_instances()
for reservation in response['Reservations']:
    for instance in reservation['Instances']:
        try:
            print(instance['KeyName'], instance['LaunchTime'], instance['PrivateIpAddress']) #instance['Tags']
        except:
            print()

minman_test 2019-12-01 23:57:29+00:00 172.31.21.164


In [ ]:
import boto3

session = boto3.session.Session(profile_name="default")
ec2 = session.resource(service_name="ec2",region_name="ap-northeast-2")

vpc = ec2.create_vpc(CidrBlock="10.105.0.0/16")
print(vpc)

vpc.create_tags(Tags=[{"Key":"Name","Value":"minman2115_vpc"}])
vpc.wait_until_available()

In [ ]:
ec2_cli = session.client(service_name="ec2",region_name="ap-northeast-2")
ec2_cli.modify_vpc_attribute(VpcId = vpc.id, EnableDnsSupport = {"Value":True})
ec2_cli.modify_vpc_attribute(VpcId = vpc.id, EnableDnsHostnames = {"Value":True})

In [ ]:
internetgateway = ec2.create_internet_gateway()
vpc.attach_internet_gateway(InternetGatewayId=internetgateway.id)

internetgateway.create_tags(Tags=[{"Key":"Name","Value":"minman2115_igw"}])

In [ ]:
routetable = vpc.create_route_table()
route = routetable.create_route(DestinationCidrBlock='0.0.0.0/0',GatewayId=internetgateway.id)

routetable.create_tags(Tags=[{"Key":"Name","Value":"minman2115_rt"}])

In [ ]:
subnet = ec2.create_subnet(CidrBlock='10.105.0.0/24',VpcId=vpc.id)
route_id = routetable.associate_with_subnet(SubnetId=subnet.id)
print(route_id)

subnet.create_tags(Tags=[{"Key":"Name","Value":"minman2115_PRzone"}])

In [ ]:
securitygroup = ec2.create_security_group(GroupName="minman2115_sg",Description='only allow SSH traffic',VpcId=vpc.id)

securitygroup.authorize_ingress(CidrIp='0.0.0.0/0',IpProtocol='tcp',FromPort=22,ToPort=22)
securitygroup.authorize_ingress(CidrIp='0.0.0.0/0',IpProtocol='tcp',FromPort=80,ToPort=80)
securitygroup.authorize_ingress(CidrIp='0.0.0.0/0',IpProtocol='tcp',FromPort=65000,ToPort=65100)

securitygroup.create_tags(Tags=[{"Key":"Name","Value":"minman2115_sg"}])

In [ ]:
outfile = open('xxxxxxxxxxx.pem','w')
print(outfile)

key_pair = ec2.create_key_pair(KeyName="xxxxxxxxxx")
print(key_pair)

KeyPairOut = str(key_pair.key_material)
outfile.write(KeyPairOut)
print(KeyPairOut)

In [ ]:
instances = ec2.create_instances(
ImageId = 'ami-0d59ddf55cdda6e21',
InstanceType='t2.micro',
MaxCount = 1,
MinCount = 1,
NetworkInterfaces=[{
'SubnetId' : subnet.id,
'DeviceIndex' : 0,
'AssociatePublicIpAddress':True,
'Groups':[securitygroup.group_id],    
}],
KeyName = 'xxxxxxxxxx')

print(instances)